# Text Classification
* **Created by:** Eric Martinez
* **For:** 3351 - AI-Powered Applications
* **At:** University of Texas Rio-Grande Valley

## Essential Components

What are the main ingredients to solving a problem well with an LLM?

- **Task:** Identify the task you are trying to solve. What are the desired inputs? What are the desired outputs
- **Data:** Identify how those inputs make their way to your system once deployed. Identify where your evaluation data will come from.
- **Metrics:** Identify the key metrics for evaluating the performance of your solution
- **Model:** What model will be used to solve this problem?

## Classification

- this is a common task where you need to apply a label to some text
- "categorizing text into buckets"

## Example Use-Cases
- automated labeling
- sentiment analysis
- automated scoring
- analyzing recording transcripts
- analyzing customer SMS messages
- analyzing incoming chatbot messages
- content filtering / safety

## Common Metrics

How can we _quantify_ the performance of our model?

#### Accuracy

Accuracy is the proportion of correct predictions among the total number of cases processed. 

#### Recall

Recall is the fraction of the positive examples that were correctly labeled by the model as positive

#### Precision

Precision is the fraction of correctly labeled positive examples out of all of the examples that were labeled as positive. 

#### F1 Score

The F1 metric is the harmonic mean of the precision and recall. It can be calculated as: F1 = 2 * (precision * recall) / (precision + recall)

## Applied Example: Spam Classification

We work for a company that is getting an increasingly large amount of spam.

Unfortunately, they cannot afford to pay for a commercial email spam detection tool.

Given the lowering and lowering cost of GPT-3.5-turbo, you suggest building a GPT based spam detection tool.

You found a tiny little dataset that you might be able to use to prototype a solution: https://huggingface.co/datasets/TrainingDataPro/email-spam-classification

#### Essential Components

Let's try to fill this out

- **Task:** 
- **Data:** 
- **Metrics:**
- **Model:**

#### Essential Components

- **Task:** 
    - Inputs: subject - text, body - text
    - Outputs: is_spam - boolean
- **Data:** 
    - Hugging Face Dataset: https://huggingface.co/datasets/TrainingDataPro/email-spam-classification
- **Metrics:**
    - Accuracy
- **Model:**
    - GPT-3.5-turbo

#### Visualize our AI model

In [1]:
from utils.magic import mermaid

In [2]:
%%mermaid
flowchart LR
subgraph Model: spam_classifier
    direction LR
    prompt_1{"Prompt\n(gpt-3.5-turbo)"}
    transform_1[[ post_process]]
    Input --> |"subject (str)"| prompt_1
    Input --> |"body (str)"| prompt_1
    prompt_1 --> |"is_spam (str)"| transform_1
    transform_1 --> |"is_spam (bool)"| Output
end

#### Spam Classifier - Data

In [ ]:
%pip install datasets

In [3]:
from datasets import load_dataset

full_dataset = load_dataset("TrainingDataPro/email-spam-classification")

print(full_dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'type'],
        num_rows: 84
    })
})


In [4]:
# Print one sample
print(full_dataset['train'][0])

{'title': '?? the secrets to SUCCESS', 'text': "Hi James,\n\nHave you claim your complimentary gift yet?\n\nI've compiled in here a special astrology gift that predicts everything about you in the future?\n\nThis is your enabler to take the correct actions now.\n\n>> Click here to claim your copy now >>\n\nClaim yours now, and thank me later.\n\n\nLove,\nHeather", 'type': 'spam'}


#### Let's load up a cleaned-up version of the dataset

Each example will have the following format:
    
```
{
    "inputs": ...
    "outputs": ...
}
```

In [5]:
from utils.example_data import load_email_spam_dataset

In [6]:
dataset = load_email_spam_dataset()
print(dataset)

{'train': [{'inputs': {'subject': 'Gut Health & Weight Loss Plan - Role of Supplement', 'body': 'Hi Maryam,,\n\nPlease find the information on the role of the supplements we take during the program.\n\nThe Supplements are high quality nutritional support assisting to Heal & Repair our Gut. The three key reasons we have the supplements are:\n\n- The primary reason is to balance your hormones � particularly your insulin levels. (Insulin being responsible for fat uptake in the cells � especially your liver cells). Once your hormones are balanced your body can in fact learn to lose weight and efficiently, and keep it off. As well as with the weight loss you will find your moods become much more stable, anxiety levels decrease, your skin becomes clearer, your head and your thoughts become clearer, and your energy levels will soar (no more naps in the afternoon � it�s very liberating).\n\n- Secondly the nutrition is responsible for cleaning your Gut. The vast majority of illnesses today stem

#### Spam Classifier - Task
- **Inputs:** subject (str), body (str)
- **Outputs:** is_spam (bool)

In [7]:
def spam_classifier(subject=None, body=None):  
    outputs = {}

    # obviously, this is not good
    outputs['is_spam'] = False
    
    return outputs

#### Spam Classifier - Metrics
    - Accuracy

In [8]:
def accuracy_metric(predictions=None, references=None):
    if not predictions:
        raise ValueError("Must supply predictions")
        
    if not references:
        raise ValueError("Must supply references")
        
    if len(predictions)!=len(references):
        raise ValueError("Length of predictions must match number of references")
        
    correct = 0
    total = len(references)
    
    for prediction, reference in zip(predictions, references):
        if prediction == reference:
            correct += 1
            
    score = (correct * 1.0) / total
    
    return score

In [9]:
# let's test out the accuracy metric
pretend_predictions = [1, 1, 0, 0]
pretend_references = [1, 1, 0, 0]
accuracy_score = accuracy_metric(predictions=pretend_predictions, references=pretend_references)
print(accuracy_score)

1.0


#### Spam Classifier - Evaluation Loop (Hand-rolled)

We need a pipeline for processing all examples and crunching metrics

In [10]:
# this is a basic implementation of an evaluation loop
predictions = []
references = []

for sample in dataset['train']:
    sample_inputs = sample['inputs']
    sample_outputs = sample['outputs']
    
    prediction = spam_classifier(**sample_inputs)
    
    predictions.append(prediction)
    references.append(sample_outputs)

    
# compute accuracy for `is_spam`
is_spam_predictions = [prediction['is_spam'] for prediction in predictions]
is_spam_references = [reference['is_spam'] for reference in references]
is_spam_accuracy_score = accuracy_metric(predictions=is_spam_predictions, references=is_spam_references)
print(f"`is_spam` accuracy score: {is_spam_accuracy_score}")

`is_spam` accuracy score: 0.6716417910447762


#### Spam Classifier - Evaluation Loop (Advanced)

We need to optimize for getting fast feedback and making it easy to experiment:
- all important code in one place, easily understandable
- start on a tiny subset of the data (10 examples)
- be able to visualize where we go wrong
- evaluation-driven development

In [14]:
from utils.example_data import load_email_spam_dataset
from utils.evaluation import evaluate
from utils.openai import chat_completion

def spam_classifier(subject=None, body=None):  
    outputs = {}
    
    # run through gpt prompt
    # ...
    
    # post-process
    # ...
    
    # set outputs, change this obv
    outputs['is_spam'] = False
    
    # return
    return outputs


# Load dataset
dataset = load_email_spam_dataset()

# Define important metrics
metrics = {
    "accuracy": {
        "function": accuracy_metric,
        "only": ["is_spam"]
    }
}

# Perform batch evaluation
results = evaluate(spam_classifier, dataset=dataset, split="train", limit=10, metrics=metrics, debug=True)

# Validate model performance
assert results['is_spam']['accuracy'] >= 0.9, "`is_spam` accuracy must be greater than or equal to 0.9 on the 'train' set"

,accuracy
is_spam,0.8


,exact_match_is_spam,prediction_is_spam,target_is_spam,input_subject,input_body
0,True,False,False,[The Virtual Reward Center] Re: ** Clarifications,"Hello,\n \nThank you for contacting the Virtual Reward Center. We here at the Virtual Reward Center do not dictate what Rewards get sent out to recipients. Please contact your program sponsor, (the company from which you originally received your Reward) at helpdesk@cspace.com for this inquiry.\n \nWe apologize for any inconvenience.\n \n \nThank You,\n\nThe Virtual Reward Center\nCustomer Support Team Lead"
1,True,False,False,"amazon.com.tr, action needed: Sign-in","\nLogo Image\nSenol Yildirim,\n\nSomeone signed-in to your account.\n\nWhen: Jul 06, 2023 07:34 PM Turkey Time\nDevice: Amazon Shopping App for Android\nNear: Manisa, Turkey\nIf this was you, you can disregard this message. Otherwise, please let us know."
2,True,False,False,Venmo primary email address changed,"venmo\nHi Maliek Folks, \n\nThe primary email address for your Venmo account was recently changed. \n\nIf this wasn�t you or you believe an unauthorized person has accessed your account, please immediately reset your password. Then, contact us by replying to this email so we can confirm your account is secure. \n\nTo reset your password tap �Forgot your password?� on the sign-in screen in the mobile app or visit www.venmo.com from a computer. \n\nThanks,\nTeam Venmo\nVenmo is a service of Pa..."
3,True,False,False,APPLICATION PROCESS,"GOOD DAY SIR/MADAM \n\n In 2020 I applied to the institution but I didn't know I was accepted for the 2021 Arcademic Year. As I was trying to apply for this year for the past few months but with no success, I noticed that I owe the institution R3280. I kindly please ask for your assistance as I wish to enroll in this Institution for this Academic year 2022.\n\nYOURS SINCERELY \nRESHOKETSWE SEKHUKHUNE"
4,True,False,False,Congratulations! You have been selected for a special scholarship from Unicaf,"Dear Joseph Alex Eze\n \nWe are pleased to inform you that as part of Unicaf�s 10-year anniversary, you have been selected for a special scholarship which will allow you to study towards a British Master�s degree of your choice for only ?1,950.\n \nWe are excited to have you join us and believe that this program will provide you with the skills and knowledge you need to succeed in your chosen field. More than 90% of our graduates are in employment or have earned a higher salary.\n \nWe belie..."
5,True,False,False,Your uploaded document is rejected,View In Browser | Log in\n \n \n\nSkrill logo\nMoney movers & makers\n\n \n?? Hi Anggraeni Dyah\nWe couldn�t verify your address\nSkrill\nDocument doesn�t meet criteria\n \nProblem: We couldn�t verify your address because your document doesn�t meet the required criteria.\nSolution: Send a proof of address that meets the requirements listed below or try sharing your location in the Skrill app.\n \n Send document \nIt�s easier to verify your address with the Skrill mobile app. Just share ...
6,True,False,False,Your Kilimall Account is Ready - Shopping Now!,"Dear Customer,\n\nWelcome to Kilimall, Thanks so much for joining us.\n\nKilimall is Africa's largest online shopping mall. We launched in July 2014 and our mission is to become the best E-commerce platform in Africa. We serve a retail customer base that continues to grow exponentially, offering products that span various categories including Phones, Computers, Clothing, Shoes, Home Appliances, Books, healthcare, Baby Products, personal care and much more.\n\nShop Now"
7,True,False,False,You applied to job post Translation_ISAAC_Acceptability_tr_TR-en_US,"Hello there!\n\nYou have started the application process for job post Acceptability - Translation Raters needed (Turkish-English) (job ID Translation_ISAAC_Acceptability_tr_TR-en_US ).\n\nYou can see the current status of your application on the My Jobs page on our user portal.\n\nPlease make sure that you complete all potential additional re

AssertionError: `is_spam` accuracy must be greater than or equal to 0.9 on the 'train' set

## Exercise 1: Complete the `spam_classifier` function above using GPT-3.5-turbo.

## Exercise 2: Create a working Gradio interface for using your model

Tips:
- use `gr.Textbox` for text entry
- you can add more lines to a textbox, for example `gr.Textbox(label="Big Box", lines=10)`
- add helpful labels to every input for user-friendliness
- you can make a Textbox non-editable by users by making it non-interactive, check the Gradio docs

In [13]:
# here is a hello world gradio application
import gradio as gr

def greet(name="Nameless"):
    greeting = f"Hello, {name}"
    return greeting
    

with gr.Blocks() as app:
    name = gr.Textbox(label="Name")
    btn = gr.Button(value="Submit")
    greeting = gr.Textbox(label="Greeting")

    btn.click(
        greet,
        inputs=[name],
        outputs=[greeting],
    )

app.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
